In [1]:
import youtube_dl
import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'youtube_dl'

In [2]:
WAV_DIR = 'wav_files/'
genre_dict = {
            '/m/064t9': 'Pop_music',
            '/m/0glt670': 'Hip_hop_music',
            '/m/06by7': 'Rock_music',
            '/m/06j6l': 'Rhythm_blues',
            '/m/06cqb': 'Reggae',
            '/m/0y4f8': 'Vocal',
            '/m/07gxw': 'Techno',
            }

genre_set = set(genre_dict.keys())

In [3]:
temp_str = []
with open('data-files/csv_files/unbalanced_train_segments.csv', 'r') as f:
    temp_str = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'data-files/csv_files/unbalanced_train_segments.csv'

In [4]:
data = np.ones(shape=(1,4)) 
for line in tqdm(temp_str):
    line = re.sub('\s?"', '', line.strip())
    elements = line.split(',')
    common_elements = list(genre_set.intersection(elements[3:]))
    if  common_elements != []:
        data = np.vstack([data, np.array(elements[:3]
                                         + [genre_dict[common_elements[0]]]).reshape(1, 4)])

df = pd.DataFrame(data[1:], columns=['url', 'start_time', 'end_time', 'class_label'])

100%|██████████████████████████████████████████████████████████████████████| 2041792/2041792 [08:09<00:00, 4170.74it/s]


In [5]:
df.tail()

url start_time  end_time class_label
52102  zzKhslmovrU     40.000    50.000      Techno
52103  zzS9dTTSMvE    190.000   200.000  Rock_music
52104  zzVw6ZHHeHo    150.000   160.000  Rock_music
52105  zzXUnHtUXvg    150.000   160.000  Rock_music
52106  zzfj1xDOIkM     10.000    20.000      Reggae

In [6]:
# Remove 10k Techno audio clips - to make the data more balanced
np.random.seed(10)
drop_indices = np.random.choice(df[df['class_label'] == 'Techno'].index, size=10000, replace=False)
df.drop(labels=drop_indices, axis=0, inplace=True)
df.reset_index(drop=True, inplace=False)

# Time to INT 
df['start_time'] = df['start_time'].map(lambda x: np.int32(np.float(x)))
df['end_time'] = df['end_time'].map(lambda x: np.int32(np.float(x)))

In [7]:
df['class_label'].value_counts()

Pop_music        8471
Rock_music       8288
Hip_hop_music    7148
Techno           7098
Rhythm_blues     4460
Vocal            3535
Reggae           3107
Name: class_label, dtype: int64

In [8]:
for i, row in tqdm(df.iterrows()):
    url = "'https://www.youtube.com/embed/" + row['url'] + "'"
    file_name = str(i)+"_"+row['class_label']
    
    try:
        command_1 = "ffmpeg -ss " + str(row['start_time']) + " -i $(youtube-dl -f 140 --get-url " +\
                    url + ") -t 10 -c:v copy -c:a copy " + file_name + ".mp4"

        command_2 = "ffmpeg -i "+ file_name +".mp4 -vn -acodec pcm_s16le -ar 44100 -ac 1 " + WAV_DIR + file_name + ".wav"

        command_3 = 'rm ' + file_name + '.mp4' 

        # Run the 3 commands
        os.system(command_1 + ';' + command_2 + ';' + command_3 + ';')
    
    except:
        print(i, url)
        pass

42107it [23:07, 30.35it/s]
